In [1]:
!pip install --upgrade --force-reinstall --no-deps kaggle

     |████████████████████████████████| 61kB 2.2MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.6-cp36-none-any.whl size=72859 sha256=c201741c12c85268b992047c495cc8d2f4d3761af75b4d4dd736da5adc08a8c1
  Stored in directory: /root/.cache/pip/wheels/57/4e/e8/bb28d035162fb8f17f8ca5d42c3230e284c6aa565b42b72674
Successfully built kaggle
  Found existing installation: kaggle 1.5.6
    Uninstalling kaggle-1.5.6:
      Successfully uninstalled kaggle-1.5.6


In [2]:
import os
os.environ['KAGGLE_USERNAME'] = " " # enter username from the kaggle json file
os.environ['KAGGLE_KEY'] = " " # enter key from the kaggle json file
!kaggle competitions download -c spooky-author-identification

  0% 0.00/1.81M [00:00<?, ?B/s]
100% 1.81M/1.81M [00:00<00:00, 153MB/s]


In [3]:
!unzip /content/spooky-author-identification.zip

Archive:  /content/spooky-author-identification.zip
  inflating: sample_submission.zip   
  inflating: test.zip                
  inflating: train.zip               


In [7]:
!unzip /content/train.zip
!unzip /content/test.zip
!unzip /content/sample_submission.zip

Archive:  /content/train.zip
  inflating: train.csv               
Archive:  /content/test.zip
  inflating: test.csv                
Archive:  /content/sample_submission.zip
  inflating: sample_submission.csv   


In [0]:
import numpy as np 
import pandas as pd 
import os

In [9]:
train = pd.read_csv('/content/train.csv')
test = pd.read_csv('/content/test.csv')
train.head()

,id,text,author
0,id26305,"This process, however, afforded me no means of...",EAP
1,id17569,It never once occurred to me that the fumbling...,HPL
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP
3,id27763,How lovely is spring As we looked from Windsor...,MWS
4,id12958,"Finding nothing else, not even gold, the Super...",HPL


In [10]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import np_utils
import numpy as np
from nltk.corpus import stopwords
from tqdm import tqdm
from string import punctuation

Using TensorFlow backend.


In [12]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
corpus = list(train['text'].values)
sw = stopwords.words('english')
vocab1 = list()
vocab = list()
for i in corpus:
    for j in punctuation:
        i = i.replace(j,'')
    vocab1.append(i)
vocab1 = [i.split() for i in vocab1]
for i in range(len(vocab1)):
    for k in sw:
        temp = [j for j in vocab1[i] if j not in sw]
    vocab.append(temp)


In [14]:
z = [len(i) for i in vocab]
max_len = np.max(z)
max_words = 15000
max_len

462

In [0]:
tk = Tokenizer(num_words=max_words)
tk.fit_on_texts(vocab)
train_encoded = tk.texts_to_sequences(vocab)
test_encoded = tk.texts_to_sequences(test['text'].values)
x_train = pad_sequences(train_encoded, max_len)
x_test = pad_sequences(test_encoded, max_len)

In [16]:
print(vocab[0],'\n', train_encoded[0],'\n', tk.word_index['however'])

['This', 'process', 'however', 'afforded', 'means', 'ascertaining', 'dimensions', 'dungeon', 'I', 'might', 'make', 'circuit', 'return', 'point', 'whence', 'I', 'set', 'without', 'aware', 'fact', 'perfectly', 'uniform', 'seemed', 'wall'] 
 [41, 2903, 62, 1307, 207, 7412, 2399, 4520, 1, 15, 98, 4211, 208, 162, 1886, 1, 240, 52, 818, 226, 1375, 4894, 27, 345] 
 62


In [0]:
labels_train = pd.get_dummies(train['author'])
y_train = []
for i in range(len(labels_train)):
    temp = [labels_train['EAP'][i], labels_train['HPL'][i], labels_train['MWS'][i]]
    y_train.append(temp)

In [0]:
x_train = np.array(x_train)
y_train = np.array(y_train)

In [0]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Activation, Embedding
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [22]:
model = Sequential()
model.add(Embedding(max_words, 512, input_length=max_len))
model.add(LSTM(128))
model.add(Dropout(0.5))
model.add(Dense(3))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 462, 512)          7680000   
_________________________________________________________________
lstm_3 (LSTM)                (None, 128)               328192    
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 387       
_________________________________________________________________
activation_3 (Activation)    (None, 3)                 0         
Total params: 8,008,579
Trainable params: 8,008,579
Non-trainable params: 0
_________________________________________________________________


In [24]:
checkpoint = ModelCheckpoint('weights.hdf5',save_best_only=True, monitor='val_loss', mode='auto', verbose=1)
earlystop = EarlyStopping(monitor='val_loss', mode='auto', patience=20)
model.fit(x_train, y_train, epochs=5, batch_size=100, callbacks=[checkpoint,earlystop], verbose=1, validation_split=0.1)

Train on 17621 samples, validate on 1958 samples
Epoch 1/5
17621/17621 [==============================] - 645s 37ms/step - loss: 0.6780 - acc: 0.7043 - val_loss: 0.4567 - val_acc: 0.8161

Epoch 00001: val_loss improved from inf to 0.45669, saving model to weights.hdf5
Epoch 2/5
17621/17621 [==============================] - 648s 37ms/step - loss: 0.2461 - acc: 0.9104 - val_loss: 0.4661 - val_acc: 0.8253

Epoch 00002: val_loss did not improve from 0.45669
Epoch 3/5
17621/17621 [==============================] - 641s 36ms/step - loss: 0.1287 - acc: 0.9564 - val_loss: 0.6017 - val_acc: 0.8095

Epoch 00003: val_loss did not improve from 0.45669
Epoch 4/5
17621/17621 [==============================] - 641s 36ms/step - loss: 0.0791 - acc: 0.9720 - val_loss: 0.7182 - val_acc: 0.8069

Epoch 00004: val_loss did not improve from 0.45669
Epoch 5/5
17621/17621 [==============================] - 645s 37ms/step - loss: 0.0566 - acc: 0.9809 - val_loss: 0.8205 - val_acc: 0.8080

Epoch 00005: val_loss 

In [0]:
model.load_weights('weights.hdf5')

In [0]:
pred = model.predict(x_test)

In [0]:
id = test['id'].values
eap = pred[:,0]
hpl = pred[:,1]
mws = pred[:,2]
submission = {'id':id, 'EAP':eap, 'HPL':hpl, 'MWS':mws}
submission = pd.DataFrame(submission)
submission.to_csv('Submission.csv',index=False)